In [ ]:
FLOWERS_DIR = './flower_photos'
TRAIN_FRACTION = 0.8
RANDOM_SEED = 2018

# デバイスを自動決定（MPS、CUDA、またはCPU）
import torch
if torch.backends.mps.is_available():
    DEVICE = "mps"
elif torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"
print(f"使用デバイス: {DEVICE}")

使用デバイス: mps


In [28]:
import urllib
import os
import random
import collections
from ultralytics import YOLO
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as T
from tqdm import tqdm

# Grounding-DINOのインポート（利用可能な場合）
try:
    from groundingdino.util.inference import load_model, load_image, predict, annotate
    import groundingdino.datasets.transforms as TS
    GROUNDING_DINO_AVAILABLE = True
    print("✓ Grounding-DINOが利用可能です")
except ImportError as e:
    print("警告: Grounding-DINOが利用できません。インストールしてください:")
    print("  git clone https://github.com/IDEA-Research/GroundingDINO.git")
    print("  cd GroundingDINO")
    print("  uv pip install -e .")
    print(f"  エラー詳細: {e}")
    GROUNDING_DINO_AVAILABLE = False



✓ Grounding-DINOが利用可能です


In [11]:
def download_images():
  """If the images aren't already downloaded, save them to FLOWERS_DIR."""
  if not os.path.exists(FLOWERS_DIR):
    DOWNLOAD_URL = 'http://download.tensorflow.org/example_images/flower_photos.tgz'
    print('Downloading flower images from %s...' % DOWNLOAD_URL)
    urllib.request.urlretrieve(DOWNLOAD_URL, 'flower_photos.tgz')
    !tar xfz flower_photos.tgz
  print('Flower photos are located in %s' % FLOWERS_DIR)


In [4]:
download_images()

Flower photos are located in ./flower_photos


In [22]:
# flowers_photos/ ディレクトリの内容を、YOLOの学習用ディレクトリに移動する。
import shutil
from pathlib import Path

# クラス名のマッピング（ディレクトリ名 -> クラスID）
CLASS_MAPPING = {
    'daisy': 0,
    'dandelion': 1,
    'roses': 2,
    'sunflowers': 3,
    'tulips': 4
}

# Grounding-DINOモデルのロード（初回のみ）
GROUNDING_DINO_CONFIG_PATH = "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
GROUNDING_DINO_CHECKPOINT_PATH = "groundingdino_swint_ogc.pth"
grounding_dino_model = None

def load_grounding_dino():
    """Grounding-DINOモデルをロード"""
    global grounding_dino_model
    
    if not GROUNDING_DINO_AVAILABLE:
        return None
    
    if grounding_dino_model is None:
        try:
            # 設定ファイルとチェックポイントのパスを確認
            config_path = GROUNDING_DINO_CONFIG_PATH
            checkpoint_path = GROUNDING_DINO_CHECKPOINT_PATH
            
            # パスが存在しない場合は、デフォルトパスを試す
            if not os.path.exists(config_path):
                config_path = "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
            if not os.path.exists(checkpoint_path):
                checkpoint_path = "groundingdino_swint_ogc.pth"
                if not os.path.exists(checkpoint_path):
                    checkpoint_path = "GroundingDINO/weights/groundingdino_swint_ogc.pth"
            
            grounding_dino_model = load_model(config_path, checkpoint_path)
            print("Grounding-DINOモデルをロードしました")
        except Exception as e:
            print(f"Grounding-DINOモデルのロードに失敗しました: {e}")
            print("groundingdino_swint_ogc.pthをダウンロードしてください")
            print("  wget https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth")
            return None
    return grounding_dino_model

def get_annotation_with_grounding_dino(image_path, class_name, class_id, debug=False):
    """Grounding-DINOを使用してバウンディングボックスを取得
    
    Args:
        image_path: 画像のパス
        class_name: クラス名（プロンプトとして使用）
        class_id: クラスID
        debug: デバッグ情報を表示するか
    
    Returns:
        (success, bbox) - successがTrueの場合、bboxはYOLO形式の正規化座標 [center_x, center_y, width, height]
                         失敗した場合はNone
    """
    if not GROUNDING_DINO_AVAILABLE:
        return False, None
    
    model = load_grounding_dino()
    if model is None:
        return False, None
    
    try:
        # 画像を読み込み
        image_source, image = load_image(str(image_path))
        
        # テキストプロンプト（複数の形式を試す）
        # Grounding-DINOは "object ." の形式を好む
        text_prompt = f"{class_name} ."
        box_threshold = 0.15  # より低いしきい値に変更
        text_threshold = 0.10  # より低いしきい値に変更
        
        # 予測（DEVICE定数を使用）
        boxes, logits, phrases = predict(
            model=model,
            image=image,
            caption=text_prompt,
            box_threshold=box_threshold,
            text_threshold=text_threshold,
            device=DEVICE
        )
        
        if debug:
            print(f"  Debug: {image_path.name} - boxes: {len(boxes)}, phrases: {phrases[:3] if len(phrases) > 0 else 'none'}")
        
        # バウンディングボックスが取得できた場合
        if len(boxes) > 0:
            # 最も信頼度が高いバウンディングボックスを選択
            best_idx = logits.argmax().item()
            box = boxes[best_idx].cpu().numpy()
            logit = logits[best_idx].item()
            
            if debug:
                print(f"  Debug: best box={box}, logit={logit:.3f}")
            
            # Grounding-DINOはcxcywh形式（center_x, center_y, width, height）を返す
            # これは既にYOLO形式と同じ
            center_x, center_y, width, height = box
            
            # 座標が有効な範囲内にあることを確認
            if 0 <= center_x <= 1 and 0 <= center_y <= 1 and 0 < width <= 1 and 0 < height <= 1:
                return True, [center_x, center_y, width, height]
            elif debug:
                print(f"  Debug: 座標が範囲外 - center_x={center_x:.3f}, center_y={center_y:.3f}, width={width:.3f}, height={height:.3f}")
        
        # バウンディングボックスが取得できなかった場合
        if debug:
            print(f"  Debug: バウンディングボックスが見つかりませんでした")
        return False, None
        
    except Exception as e:
        # デバッグモードの場合はエラーを表示
        if debug:
            print(f"  Error: {image_path.name} - {e}")
            import traceback
            traceback.print_exc()
        return False, None

def organize_yolo_dataset():
    """flower_photosディレクトリの内容をYOLOの学習形式に変換
    
    - 各ディレクトリ（daisy, dandelion, roses, sunflowers, tulips）ごとにクラス番号を割り振る
    - train/valに分割して移動
    - 各画像に対応するラベルテキストファイルを作成（アノテーション範囲は画面の全領域）
    """
    
    # ランダムシードを設定
    random.seed(RANDOM_SEED)
    
    # 出力ディレクトリを作成（dataset/以下に配置）
    train_images_dir = Path('dataset/train/images')
    train_labels_dir = Path('dataset/train/labels')
    val_images_dir = Path('dataset/val/images')
    val_labels_dir = Path('dataset/val/labels')
    
    for dir_path in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
        dir_path.mkdir(parents=True, exist_ok=True)
    
    # Grounding-DINOモデルをロード
    print("Grounding-DINOモデルをロード中...")
    load_grounding_dino()
    
    # 各クラスディレクトリを処理
    flowers_path = Path(FLOWERS_DIR)
    total_images = 0
    train_count = 0
    val_count = 0
    annotation_success_count = 0
    annotation_fail_count = 0
    
    for class_name, class_id in CLASS_MAPPING.items():
        class_dir = flowers_path / class_name
        
        if not class_dir.exists():
            print(f"警告: {class_dir} が見つかりません。スキップします。")
            continue
        
        # 画像ファイルを取得
        image_files = list(class_dir.glob('*.jpg'))
        total_images += len(image_files)
        
        # ランダムにシャッフル
        random.shuffle(image_files)
        
        # 成功した画像と失敗した画像を分けて処理
        success_images = []  # (img_path, bbox) のリスト
        fail_images = []  # img_path のリスト
        
        # 各画像を処理してアノテーションを取得（進捗バー付き）
        print(f"\n{class_name} (クラスID: {class_id}): アノテーション取得中...")
        for idx, img_path in enumerate(tqdm(image_files, desc=f"  {class_name}", unit="画像")):
            # 最初の数枚の画像でデバッグ情報を表示
            debug_mode = (idx < 3) and (class_name == list(CLASS_MAPPING.keys())[0])
            
            # Grounding-DINOでアノテーションを取得
            success, bbox = get_annotation_with_grounding_dino(img_path, class_name, class_id, debug=debug_mode)
            
            if success:
                success_images.append((img_path, bbox))
                annotation_success_count += 1
            else:
                fail_images.append(img_path)
                annotation_fail_count += 1
        
        # 成功した画像をtrain/valに分割
        random.shuffle(success_images)
        split_idx = int(len(success_images) * TRAIN_FRACTION)
        success_train = success_images[:split_idx]
        success_val = success_images[split_idx:]
        
        # 成功した画像のtrain分を処理（進捗バー付き）
        if len(success_train) > 0:
            print(f"  train用画像をコピー中... ({len(success_train)} 画像)")
            for img_path, bbox in tqdm(success_train, desc="    train", unit="画像", leave=False):
                dest_img = train_images_dir / img_path.name
                shutil.copy2(img_path, dest_img)
                
                # ラベルファイルを作成
                label_path = train_labels_dir / (img_path.stem + '.txt')
                with open(label_path, 'w') as f:
                    # YOLO形式: class_id center_x center_y width height (正規化座標)
                    f.write(f"{class_id} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")
                
                train_count += 1
        
        # 成功した画像のval分を処理（進捗バー付き）
        if len(success_val) > 0:
            print(f"  val用画像をコピー中... ({len(success_val)} 画像)")
            for img_path, bbox in tqdm(success_val, desc="    val", unit="画像", leave=False):
                dest_img = val_images_dir / img_path.name
                shutil.copy2(img_path, dest_img)
                
                # ラベルファイルを作成（Grounding-DINOで取得したアノテーションを使用）
                label_path = val_labels_dir / (img_path.stem + '.txt')
                with open(label_path, 'w') as f:
                    f.write(f"{class_id} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")
                
                val_count += 1
        
        # 失敗した画像は全てvalに追加（画像全体を範囲として）（進捗バー付き）
        if len(fail_images) > 0:
            print(f"  失敗画像をvalにコピー中... ({len(fail_images)} 画像)")
            for img_path in tqdm(fail_images, desc="    val(fail)", unit="画像", leave=False):
                dest_img = val_images_dir / img_path.name
                shutil.copy2(img_path, dest_img)
                
                # ラベルファイルを作成（アノテーション範囲は画面の全領域）
                label_path = val_labels_dir / (img_path.stem + '.txt')
                with open(label_path, 'w') as f:
                    f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")
                
                val_count += 1
        
        print(f"  {class_name} (クラスID: {class_id}): 処理完了 ({len(image_files)} 画像)")
    
    print(f"\n完了!")
    print(f"総画像数: {total_images}")
    print(f"train: {train_count} 画像")
    print(f"val: {val_count} 画像")
    print(f"\nアノテーション統計:")
    print(f"  成功: {annotation_success_count} 画像 (うち {train_count} 画像をtrain、{annotation_success_count - train_count} 画像をvalに配置)")
    print(f"  失敗: {annotation_fail_count} 画像 (全てvalに配置、画像全体を範囲として使用)")
    print(f"\nディレクトリ構造:")
    print(f"  dataset/train/images/ - {len(list(train_images_dir.glob('*.jpg')))} 画像")
    print(f"  dataset/train/labels/ - {len(list(train_labels_dir.glob('*.txt')))} ラベル")
    print(f"  dataset/val/images/ - {len(list(val_images_dir.glob('*.jpg')))} 画像")
    print(f"  dataset/val/labels/ - {len(list(val_labels_dir.glob('*.txt')))} ラベル")

# 実行
organize_yolo_dataset()


Grounding-DINOモデルをロード中...
final text_encoder_type: bert-base-uncased
Grounding-DINOモデルをロードしました

daisy (クラスID: 0): アノテーション取得中...


  daisy:   0%|          | 1/633 [00:02<24:22,  2.31s/画像]

  Debug: 20580471306_ab5a011b15_n.jpg - boxes: 8, phrases: ['daisy', 'daisy', 'daisy']
  Debug: best box=[    0.54213     0.49349     0.91572     0.75768], logit=0.553


  daisy:   0%|          | 2/633 [00:02<13:49,  1.31s/画像]

  Debug: 7227973870_806d9d3e42_n.jpg - boxes: 17, phrases: ['daisy', 'daisy', 'daisy']
  Debug: best box=[     0.6208      0.4727     0.50358     0.69881], logit=0.564


...

  sunflowers (クラスID: 3): 処理完了 (699 画像)

tulips (クラスID: 4): アノテーション取得中...


  tulips: 100%|██████████| 799/799 [09:42<00:00,  1.37画像/s]


  train用画像をコピー中... (639 画像)


  val用画像をコピー中... (160 画像)


  tulips (クラスID: 4): 処理完了 (799 画像)

完了!
総画像数: 3670
train: 2934 画像
val: 736 画像

アノテーション統計:
  成功: 3670 画像 (うち 2934 画像をtrain、736 画像をvalに配置)
  失敗: 0 画像 (全てvalに配置、画像全体を範囲として使用)

ディレクトリ構造:
  dataset/train/images/ - 2933 画像
  dataset/train/labels/ - 2933 ラベル
  dataset/val/images/ - 736 画像
  dataset/val/labels/ - 736 ラベル


In [24]:
model = YOLO('yolov8n.pt')

results = model.train(
    data=r'/Users/ikomiki/workspace-outer/YOLO-flower/data.yaml',  # データセットの設定ファイル
    epochs=10,  # 学習回数
    batch=8,  # バッチサイズ
    imgsz=640,  # 画像サイズ（入力画像をリサイズ）
    device=DEVICE  # CPU 又は GPU（CUDA）
)

Ultralytics 8.3.229 🚀 Python-3.12.7 torch-2.9.1 MPS (Apple M4 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/ikomiki/workspace-outer/YOLO-flower/data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True,

In [25]:
print(results)

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x86d813530>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [27]:

# 追加学習後のモデルをロード（学習済みモデルのパスを指定）
model = YOLO(results.save_dir / 'weights' / 'best.pt')  # "best.pt" は最も精度が高い学習済みモデル

# 検証を実施
results = model.val(
    data=r'/Users/ikomiki/workspace-outer/YOLO-flower/data.yaml',  # データセットの設定ファイル
    batch=8,  # バッチサイズ
    imgsz=640,  # 画像サイズ（入力画像をリサイズ）
    device=DEVICE,  # CPU または GPU（CUDA）
)

Ultralytics 8.3.229 🚀 Python-3.12.7 torch-2.9.1 MPS (Apple M4 Pro)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 186.0±76.2 MB/s, size: 64.3 KB)
val: Scanning /Users/ikomiki/workspace-outer/YOLO-flower/dataset/val/labels.cache... 736 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 736/736 1.5Mit/s 0.0ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 92/92 2.9it/s 31.6s0.2s
                   all        736        736      0.711      0.683      0.741      0.658
                 daisy        127        127      0.725      0.706      0.777        0.7
             dandelion        180        180      0.825      0.839      0.907       0.81
                 roses        129        129      0.649      0.529      0.596      0.496
            sunflowers        140        140      0.655      0.721      0.693      0.644
                tulips        